<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/merge_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import shutil

In [2]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [6]:
a = pd.read_csv('/content/drive/MyDrive/wifi_train_timestamp_test_path/5a0546857ecc773753327266_train.csv')
a

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd76579680a8142a19e69dcd6c33db61dc0a2444,fd7e06b88954f0ca0a2987e1741453624462bcc6,fd83a61cb82476f0ad6af28c4cd5872f285fd104,fd963f68e158c7e77bdd1d0a867927a1d270e972,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,x,y,f,path,timestamp
0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-92,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-51,-999,-999,-999,-999,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,41.316772,180.01710,-1,5e1580bb1506f2000638fc62,1578466888412
1,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-92,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-54,-999,-999,-999,-999,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,47.679752,178.77545,-1,5e1580bb1506f2000638fc62,1578466890352
2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-99

In [4]:
beacon = pd.read_csv('/content/drive/MyDrive/beacon_distance/5a0546857ecc773753327266_train.csv')
beacon

,Unnamed: 0,002989774ac3f7cbe235b1d64624b86ff2de44cc,0038c3e64749139268d6f6af5edb7717f1d9c431,005e9a769a3de05abf218233ae9865b9d1744aa8,008be9d228e0defc111674a606c24538209f59cd,0092c5464b1edf71b38921aa1f9447f71fb3e20d,013b65bdef99464b5e84e30d2a225db6158f803b,0177a4572decd95abe4b7ce0233b054b4e2a3090,01b232cb492cd05b62dbb7c9e6ddedb64f07d2ec,028bce0f2f389bddfb6642eb85c4153dc036ce83,033b939c3b4bac271255f7d984dc0dea3670b0d2,03759fa6d12a590473ec3cf3d208cf87cd61cfcb,03f3cb0e97dd3ba9bdb00ed88fcea65e7ac6ac42,04abedb1e03323ed2e52b7d5042f1434e473513e,04dd84bc0dd1033cd20c7d221351aa32eaad801a,04e1f04e3dc6bfbca6a3ffc38bf1d4bbdae64d58,0550cf900774e946d2cbdf0de8bd1979ecc98d5f,05958186eef1d4df1e84b485bae2a463ff71a2de,05a55c81bb2fe374d0e796365b82d87ec8b54106,05f0d50041aecc5ef62b99fa1f0eb23e4a600b9d,071e2639e1726e1779259a9a26a1605d04ced70f,071ec59b6d4fd5f22a81cdc258844ca0f89231b5,0746e8c2333070f64f1131b52b6cacda7fa2efca,077aff11475853ae1f71741e76d6a5dff1e3bebe,07d600de67cd3b8f1f09e3a5a70f0dcfda60a034,0829936b5df907e90aca31b0b5cf8ffdc47e2740,0843bcefa3e66481cf0b609355f495972d3d134e,087e9e4b480c3f9fa3d68e5a36bac45a5a7a6b20,08b15360e96194c6e9b2b561bebe32f3806d11ba,09632c0b7fab73d78730d373460c12fd318ce377,09c16ad1723d9964dbe515b3a80a96ae58cb4595,09e406d4182c10357b390815a86b51846f979b6d,09e4396f9c49c2762c44642d9bf6ddf31c3fa66c,0a26b82d1b6ba5d46704e17f1cb1a310634eaf87,0b54b2d78e215e868795fb742f17655b1d751dbd,0c52783b8d30536c06309e54c97cc5f455e3df4d,0ccdd273f55badac16bb01a66cf26888a9745740,0d08642377afb5ba7f4988febf1226f69a5f44bf,0d11945a5e0dc6e0384456a24ad96efa25b825c1,0d223d74a6b81d26bb70389f6c171f59d283fe77,...,f7004a6cfa4c6f84b90231ac74129fdb180bb189,f71165af62d9b61987dc6ff033fa41b52316e887,f731e5324163f9f00d774d2c42f5eb44135064b4,f73cb58fb4e4887813453f8886a947d5c8528871,f7532d588fedcd69d92dfb69a93940e2d9d259cd,f8757362b2d4e68fd3a70edaf05ca2d7b720faec,f8b80b08661e8f03d76d18a45bb6a073f2d4a382,f8eeb01928dc5b9b8e010146a919a02985b9397a,f9174cce350bb73826a93f541826b28ed313d247,f98b5eaca351132a356e1759afeec450d4db5047,fa04592b83f51b4a116d0a7d15114526301b82fb,fa220899c86417ecdee177be6b8b3833df479e61,fa4d1a4bb2c1856ebc95837f19bcedeaed7afe5c,fb379c0cbe2f6d80f898c89c77fc485884db12c8,fb9542240afba0385a86ac3a66e332ff79ba4517,fbe004d6f30ef6151a4bb8798621237b14d122ef,fc23c34f0c650cedf51e0a56996bc06855193d87,fc65e1ebdd6d3cbe8af8f05bfe3746a27a8f8650,fc6854f981fb65feaf6eb503d951bf452d93f7dc,fc7cd7da03d88249e7f4042ad1576d5a06b9a601,fcd73ad407879a0a28249112660ab85f1d2ca7bc,fd9884c76d240b400d9edce83e4eb112520ce4ab,fdb4f1627ff1c12ef9f95ea35179272a9b84a787,fdbe698b1a77cf660f2e2e01314c5c310c8fc049,fddc27a19367e6ea018025fe71cb4afeab7d46fa,fdec5c6b40dfa1ec76e00eda5842b85a48bf8167,fe5008b4413838b9435a9e706c79ed54b0839e33,ff70ebd18ad0086832fc678eae41024e33400258,ff9a29cb2dcb4100880d02b305d5691e578e66cf,ffa010feddf16dcf412f30c9e00315b2ccad77cb,ffa3bb5b0e863fbc01062a7a600c9cb1b545b08c,ffcd16a3bce1bffc95bcb26534330effa33a0f87,ffdc94c22827a4971398153ee4d7d1a2dc12b875,fff4fbb93b7d19f996738f549c0d1d8f459c05ce,fffc54a8cd43f898fb269bd9299ac133d37e19fb,x,y,f,path,timestamp
0,7,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,41.316772,180.01710,-1,5e1580bb1506f2000638fc62,1578466886571
1,7,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999

# trainとsensor結合

In [13]:
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv').drop('Unnamed: 0', axis=1)
    way_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv').drop(['Unnamed: 0', 'y'], axis=1).sort_values('timestamp')
    acc_x_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/x/{building}_train.csv').drop(['Unnamed: 0','path'], axis=1)
    acc_y_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/y/{building}_train.csv').drop(['Unnamed: 0','path', 'timestamp'], axis=1)
    acc_z_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/z/{building}_train.csv').drop(['Unnamed: 0','path', 'timestamp'], axis=1)
    gyro_x_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/x/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_y_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/y/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_z_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/z/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_x_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/x/{building}_train.csv').drop(['Unnamed: 0', 'path_id', 'timestamp'], axis=1)
    mag_y_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/y/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_z_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/z/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_x_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/x/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_y_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/y/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_z_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/z/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    #trainとwaypointをmerge
    way_df = way_df.rename(columns={'timestamp':'waypoint_timestamp'})
    sensor_df = pd.concat([acc_x_df, acc_y_df, acc_z_df,
                           gyro_x_df, gyro_y_df, gyro_z_df,
                           mag_x_df, mag_y_df, mag_z_df,
                           rotation_x_df, rotation_y_df, rotation_z_df], axis=1).sort_values('timestamp')
    sensor_df_ = pd.merge_asof(way_df, sensor_df, left_on='waypoint_timestamp', right_on='timestamp').drop(['timestamp', 'waypoint_timestamp'], axis=1)
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    train_sensor_dfs = []
    for path in used_path:
        train_path_df = train_df[train_df['path'] == path].sort_values('x')
        sensor_path_df = sensor_df_[sensor_df_['path_id'] == path].sort_values('x')
        train_sensor_path_df = pd.merge_asof(train_path_df, sensor_path_df, on='x')
        train_sensor_dfs.append(train_sensor_path_df)
    train_sensor_df = pd.concat(train_sensor_dfs)
    target_df = train_sensor_df.iloc[:, -606:-601]
    train_sensor_df = train_sensor_df.drop(['f', 'x', 'y', 'path', 'timestamp', 'path_id'], axis=1)
    train_sensor_df = pd.concat([train_sensor_df, target_df], axis=1)
    train_sensor_df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/all_data_50_02')
    print(building)

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [11]:
train_sensor_df

,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,0300016be943084d45c80e9190ab89792f2e6386,...,rotation_z_15,rotation_z_16,rotation_z_17,rotation_z_18,rotation_z_19,rotation_z_20,rotation_z_21,rotation_z_22,rotation_z_23,rotation_z_24,rotation_z_25,rotation_z_26,rotation_z_27,rotation_z_28,rotation_z_29,rotation_z_30,rotation_z_31,rotation_z_32,rotation_z_33,rotation_z_34,rotation_z_35,rotation_z_36,rotation_z_37,rotation_z_38,rotation_z_39,rotation_z_40,rotation_z_41,rotation_z_42,rotation_z_43,rotation_z_44,rotation_z_45,rotation_z_46,rotation_z_47,rotation_z_48,rotation_z_49,x,y,f,path,timestamp
0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,0.981790,0.982691,0.980680,0.982584,0.981948,0.985302,0.991331,-0.993256,-0.992934,-0.994244,-0.992265,-0.987843,-0.985639,-0.985558,-0.983341,-0.981108,-0.976773,-0.977126,-0.973437,-0.970938,-0.975780,-0.978784,-0.980305,-0.979842,-0.982146,-0.980399,-0.981278,-0.984117,-0.988663,-0.987485,-0.991021,-0.994774,-0.995488,-0.995609,-0.993628,76.25184,30.51084,2,5d10a1669c50c70008fe8977,1561368780869
1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,0.981790,0.982691,0.980680,0.982584,0.981948,0.985302,0.991331,-0.993256,-0.992934,-0.994244,-0.992265,-0.987843,-0.985639,-0.985558,-0.983341,-0.981108,-0.976773,-0.977126,-0.973437,-0.970938,-0.975780,-0.978784,-0.980305,-0.979842,-0.982146,-0.980399,-0.981278,-0.984117,-0.988663,-0.987485,-0.991021,-0.994774,-0.995488,-0.995609,-0.993628,76.25184,30.51084,2,5d10a1669c50c70008fe8977,1561368782895
2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,0.981790,0.982691,0.980680,0.982584,0.981948,0.985302,0.991331,-0.993256,-0.992934,-0.994244,-0.992265,-0.987843,-0.985639,-0.985558,-0.983341,-0.981108,-0.976773,-0.977126,-0.973437,-0.970938,-0.975780,-0.978784,-0.980305,-0.979842,-0.982146,-0.980399,-0.981278,-0.984117,-0.988663,-0.987485,-0.991021,-0.994774,-0.995488,-0.995609,-0.993628,76.25184,30.51084,2,5d10a1669c50c70008fe8977,1561368768807
3,-999,-999,-999,-99

In [104]:
train_sensor_df

,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,0300016be943084d45c80e9190ab89792f2e6386,...,rotation_z_15,rotation_z_16,rotation_z_17,rotation_z_18,rotation_z_19,rotation_z_20,rotation_z_21,rotation_z_22,rotation_z_23,rotation_z_24,rotation_z_25,rotation_z_26,rotation_z_27,rotation_z_28,rotation_z_29,rotation_z_30,rotation_z_31,rotation_z_32,rotation_z_33,rotation_z_34,rotation_z_35,rotation_z_36,rotation_z_37,rotation_z_38,rotation_z_39,rotation_z_40,rotation_z_41,rotation_z_42,rotation_z_43,rotation_z_44,rotation_z_45,rotation_z_46,rotation_z_47,rotation_z_48,rotation_z_49,x,y,f,path,timestamp
0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,0.981790,0.982691,0.980680,0.982584,0.981948,0.985302,0.991331,-0.993256,-0.992934,-0.994244,-0.992265,-0.987843,-0.985639,-0.985558,-0.983341,-0.981108,-0.976773,-0.977126,-0.973437,-0.970938,-0.975780,-0.978784,-0.980305,-0.979842,-0.982146,-0.980399,-0.981278,-0.984117,-0.988663,-0.987485,-0.991021,-0.994774,-0.995488,-0.995609,-0.993628,76.25184,30.51084,2,5d10a1669c50c70008fe8977,1561368780869
1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,0.981790,0.982691,0.980680,0.982584,0.981948,0.985302,0.991331,-0.993256,-0.992934,-0.994244,-0.992265,-0.987843,-0.985639,-0.985558,-0.983341,-0.981108,-0.976773,-0.977126,-0.973437,-0.970938,-0.975780,-0.978784,-0.980305,-0.979842,-0.982146,-0.980399,-0.981278,-0.984117,-0.988663,-0.987485,-0.991021,-0.994774,-0.995488,-0.995609,-0.993628,76.25184,30.51084,2,5d10a1669c50c70008fe8977,1561368782895
2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,0.981790,0.982691,0.980680,0.982584,0.981948,0.985302,0.991331,-0.993256,-0.992934,-0.994244,-0.992265,-0.987843,-0.985639,-0.985558,-0.983341,-0.981108,-0.976773,-0.977126,-0.973437,-0.970938,-0.975780,-0.978784,-0.980305,-0.979842,-0.982146,-0.980399,-0.981278,-0.984117,-0.988663,-0.987485,-0.991021,-0.994774,-0.995488,-0.995609,-0.993628,76.25184,30.51084,2,5d10a1669c50c70008fe8977,1561368768807
3,-999,-999,-999,-99

In [ ]:
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv').drop('Unnamed: 0', axis=1)
    way_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv').drop(['Unnamed: 0', 'y'], axis=1).sort_values('timestamp')
    acc_x_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/x/{building}_train.csv').drop(['Unnamed: 0','path'], axis=1)
    acc_y_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/y/{building}_train.csv').drop(['Unnamed: 0','path', 'timestamp'], axis=1)
    acc_z_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/z/{building}_train.csv').drop(['Unnamed: 0','path', 'timestamp'], axis=1)
    gyro_x_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/x/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_y_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/y/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_z_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/z/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_x_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/x/{building}_train.csv').drop(['Unnamed: 0', 'path_id', 'timestamp'], axis=1)
    mag_y_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/y/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_z_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/z/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_x_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/x/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_y_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/y/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_z_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/z/{building}_train.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    #trainとwaypointをmerge
    way_df = way_df.rename(columns={'timestamp':'waypoint_timestamp'})
    sensor_df = pd.concat([acc_x_df, acc_y_df, acc_z_df,
                           gyro_x_df, gyro_y_df, gyro_z_df,
                           mag_x_df, mag_y_df, mag_z_df,
                           rotation_x_df, rotation_y_df, rotation_z_df], axis=1).sort_values('timestamp')
    sensor_df_ = pd.merge_asof(way_df, sensor_df, left_on='waypoint_timestamp', right_on='timestamp').drop(['timestamp', 'waypoint_timestamp'], axis=1)
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    train_sensor_dfs = []
    for path in used_path:
        train_path_df = train_df[train_df['path'] == path].sort_values('x')
        sensor_path_df = sensor_df_[sensor_df_['path_id'] == path].sort_values('x')
        train_sensor_path_df = pd.merge_asof(train_path_df, sensor_path_df, on='x')
        train_sensor_dfs.append(train_sensor_path_df)
    train_sensor_df = pd.concat(train_sensor_dfs)
    target_df = train_sensor_df.iloc[:, -606:-601]
    train_sensor_df = train_sensor_df.drop(['f', 'x', 'y', 'path', 'timestamp', 'path_id'], axis=1)
    train_sensor_df = pd.concat([train_sensor_df, target_df], axis=1)
    train_sensor_df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/all_data_50_20')
    print(building)

In [15]:
for building in used_buildings:
    test_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_test_with_floor.csv').drop('Unnamed: 0', axis=1)
    acc_x_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/x/{building}_test.csv').drop(['Unnamed: 0'], axis=1)
    acc_y_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/y/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    acc_z_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_x_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/x/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_y_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/y/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    gyro_z_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_x_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/x/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_y_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/y/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_z_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_x_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/x/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_y_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/y/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_z_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    #trainとwaypointをmerge
    sensor_df = pd.concat([acc_x_df, acc_y_df, acc_z_df,
                           gyro_x_df, gyro_y_df, gyro_z_df,
                           mag_x_df, mag_y_df, mag_z_df,
                           rotation_x_df, rotation_y_df, rotation_z_df], axis=1)
    used_path = sorted(test_df['path'].value_counts().index.tolist())
    test_sensor_dfs = []
    for path in used_path:
        test_path_df = test_df[test_df['path'] == path].sort_values('timestamp')
        sensor_path_df = sensor_df[sensor_df['path'] == path].sort_values('timestamp').drop('path', axis=1)
        test_sensor_path_df = pd.merge_asof(test_path_df, sensor_path_df, on='timestamp', direction='nearest')
        test_sensor_dfs.append(test_sensor_path_df)
    test_sensor_df = pd.concat(test_sensor_dfs)
    target_df = test_sensor_df.iloc[:, -604:-600]
    test_sensor_df = test_sensor_df.drop(['floor', 'path', 'timestamp','site_path_timestamp'], axis=1)
    test_sensor_df = pd.concat([test_sensor_df, target_df], axis=1)
    test_sensor_df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/all_data_50_02')
    print(building)

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [112]:
test_sensor_df

,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,0300016be943084d45c80e9190ab89792f2e6386,...,rotation_z_14,rotation_z_15,rotation_z_16,rotation_z_17,rotation_z_18,rotation_z_19,rotation_z_20,rotation_z_21,rotation_z_22,rotation_z_23,rotation_z_24,rotation_z_25,rotation_z_26,rotation_z_27,rotation_z_28,rotation_z_29,rotation_z_30,rotation_z_31,rotation_z_32,rotation_z_33,rotation_z_34,rotation_z_35,rotation_z_36,rotation_z_37,rotation_z_38,rotation_z_39,rotation_z_40,rotation_z_41,rotation_z_42,rotation_z_43,rotation_z_44,rotation_z_45,rotation_z_46,rotation_z_47,rotation_z_48,rotation_z_49,floor,site_path_timestamp,path,timestamp
0,-51,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-71,-55,-999,-75,-999,-999,-999,-999,-66,-47,-999,-999,-999,-999,-999,...,0.477857,0.483148,0.472901,0.464871,0.455574,0.447010,0.454366,0.472475,0.492035,0.506834,0.504161,0.504161,0.504161,0.498559,0.486900,0.476392,0.473260,0.473269,0.466078,0.479285,0.501165,0.522963,0.544902,0.532904,0.501698,0.513921,0.507200,0.499379,0.509772,0.507073,0.513643,0.544416,0.559028,0.571771,0.577728,0.576123,0,5a0546857ecc773753327266_046cfa46be49fc1083481...,046cfa46be49fc10834815c6,9
1,-55,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-65,-999,-82,-999,-999,-999,-999,-66,-65,-999,-999,-999,-999,-999,...,0.603004,0.574760,0.540703,0.532340,0.542923,0.586869,0.600206,0.598330,0.569310,0.541568,0.525385,0.540109,0.573989,0.608517,0.603561,0.575430,0.528675,0.455099,0.362264,0.347618,0.123781,0.022384,-0.061763,-0.121092,-0.146612,-0.153667,-0.136011,-0.145176,-0.127882,-0.136113,-0.140241,-0.140846,-0.146350,-0.136724,-0.154821,-0.183343,0,5a0546857ecc773753327266_046cfa46be49fc1083481...,046cfa46be49fc10834815c6,9017
2,-50,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-78,-65,-999,-76,-999,-999,-999,-999,-999,-58,-999,-999,-999,-999,-999,...,-0.180466,-0.167868,-0.157777,-0.153837,-0.165943,-0.193560,-0.210154,-0.229311,-0.244802,-0.243998,-0.231601,-0.210144,-0.206158,-0.170119,-0.158427,-0.155377,-0.169411,-0.185260,-0.199459,-0.217468,-0.229275,-0.232931,-0.212915,-0.181617,-0.156793,-0.126817,-0.113446,-0.118332,-0.136755,-0.154561,-0.182386,-0.198651,-0.199322,-0.178371,-0.147836,-0.117947,0,5a0546857ecc773753327266_

In [109]:
test_sensor_df.iloc[:, 3390:3410]

,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,site_path_timestamp,acc_z_0,acc_z_1,acc_z_2,acc_z_3,acc_z_4,acc_z_5,acc_z_6,acc_z_7,acc_z_8,acc_z_9,acc_z_10,acc_z_11
0,-999,-71,-999,-999,-999,-999,-999,5a0546857ecc773753327266_046cfa46be49fc1083481...,7.810059,7.541689,7.645388,7.224835,10.644713,9.413268,12.923731,11.657932,6.233214,7.147763,7.376616,11.079867
1,-999,-71,-999,-89,-999,-999,-999,5a0546857ecc773753327266_046cfa46be49fc1083481...,11.867035,7.549535,10.308873,10.385222,7.478750,4.331392,12.304123,9.809507,7.172683,12.591861,10.762684,6.573620
2,-999,-71,-999,-89,-999,-999,-999,5a0546857ecc773753327266_046cfa46be49fc1083481...,5.631348,11.211360,12.927502,11.751061,5.941952,5.665984,12.016386,11.549196,13.298562,9.406809,4.812763,9.237690
3,-999,-71,-999,-89,-999,-999,-999,5a0546857ecc773753327266_046cfa46be49fc1083481...,10.250351,11.849565,10.541319,13.159689,6.936376,6.409390,3.392424,7.648950,9.358439,10.481340,8.488606,7.792973
4,-999,-71,-999,-89,-999,-999,-999,5a0546857ecc773753327266_046cfa46be49fc1083481...,11.048828,9.826854,5.981275,6.602743,7.717269,9.788683,10.357691,6.628530,4.846596,5.471032,9.677081,9.383824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,-999,-999,-999,-999,-999,-999,-999,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,12.773239,14.069470,8.356834,8.193386,12.066083,9.732135,6.077121,7.892767,12.965549,9.820589,7.904464,7.412297
5,-999,-999,-999,-999,-999,-999,-999,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,7.265381,8.670997,12.845375,10.859059,6.928023,9.823734,11.664595,9.407490,6.851798,7.724639,7.318422,8.639025
6,-999,-999,-999,-999,-999,-999,-999,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,8.261978,10.424446,15.289512,5.946676,10.953120,6.813327,3.835667,12.434956,6.637432,6.421695,12.949893,10.198702
7,-999,-999,-999,-999,-999,-999,-999,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,15.130341,6.534542,11.052624,9.808242,7.179471,4.036868,13.291014,15.181969,4.726887,10.851117,9.552361,7.107572


In [45]:
for building in used_buildings:
    test_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_test.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/acc_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path'], axis=1)
    gyro_df = pd.read_csv(f'/content/drive/MyDrive/gyro_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    mag_df = pd.read_csv(f'/content/drive/MyDrive/mag_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    rotation_df = pd.read_csv(f'/content/drive/MyDrive/rotation_50/z/{building}_test.csv').drop(['Unnamed: 0', 'path', 'timestamp'], axis=1)
    #trainとwaypointをmerge
    sensor_df = pd.concat([acc_df, gyro_df, mag_df, rotation_df], axis=1)
    used_path = sorted(test_df['path'].value_counts().index.tolist())
    test_sensor_dfs = []
    for path in used_path:
        test_path_df = test_df[test_df['path'] == path].sort_values('timestamp')
        sensor_path_df = sensor_df[sensor_df['path'] == path].sort_values('timestamp')
        test_sensor_path_df = pd.merge_asof(test_path_df, sensor_path_df, on='timestamp')
        test_sensor_dfs.append(test_sensor_path_df)
    test_sensor_df = pd.concat(test_sensor_dfs)
    test_sensor_df#.to_csv(f'{building}_test.csv')
    #shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/all_data_50')

In [43]:
test_df

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd5875bae44a87665f646e6c6e14c525b8950f6b,fd5da29ea31fc8fa2315f92f0c067e4e24e42b05,fd76579680a8142a19e69dcd6c33db61dc0a2444,fd7e06b88954f0ca0a2987e1741453624462bcc6,fd83a61cb82476f0ad6af28c4cd5872f285fd104,fd963f68e158c7e77bdd1d0a867927a1d270e972,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,site_path_timestamp,path,timestamp
0,4,-51,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-71,-55,-999,-75,-999,-999,-999,-999,-66,-47,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-83,-999,-999,-83,-999,-78,-999,-63,-999,-999,-61,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-999,-999,-999,-999,5a0546857ecc773753327266_046cfa46be49fc1083481...,046cfa46be49fc10834815c6,9
1,4,-55,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-65,-999,-82,-999,-999,-999,-999,-66,-65,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-77,-999,-999,-86,-999,-75,-999,-70,-999,-999,-57,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-89,-999,-999,-999,5a0546

In [42]:
test_sensor_df

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,rotation_z_10,rotation_z_11,rotation_z_12,rotation_z_13,rotation_z_14,rotation_z_15,rotation_z_16,rotation_z_17,rotation_z_18,rotation_z_19,rotation_z_20,rotation_z_21,rotation_z_22,rotation_z_23,rotation_z_24,rotation_z_25,rotation_z_26,rotation_z_27,rotation_z_28,rotation_z_29,rotation_z_30,rotation_z_31,rotation_z_32,rotation_z_33,rotation_z_34,rotation_z_35,rotation_z_36,rotation_z_37,rotation_z_38,rotation_z_39,rotation_z_40,rotation_z_41,rotation_z_42,rotation_z_43,rotation_z_44,rotation_z_45,rotation_z_46,rotation_z_47,rotation_z_48,rotation_z_49
0,4,-51,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-71,-55,-999,-75,-999,-999,-999,-999,-66,-47,-999,-999,-999,-999,...,0.456846,0.476322,0.486207,0.477178,0.477857,0.483148,0.472901,0.464871,0.455574,0.447010,0.454366,0.472475,0.492035,0.506834,0.504161,0.504161,0.504161,0.498559,0.486900,0.476392,0.473260,0.473269,0.466078,0.479285,0.501165,0.522963,0.544902,0.532904,0.501698,0.513921,0.507200,0.499379,0.509772,0.507073,0.513643,0.544416,0.559028,0.571771,0.577728,0.576123
1,4,-55,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-65,-999,-82,-999,-999,-999,-999,-66,-65,-999,-999,-999,-999,...,0.524416,0.548362,0.591476,0.618795,0.603004,0.574760,0.540703,0.532340,0.542923,0.586869,0.600206,0.598330,0.569310,0.541568,0.525385,0.540109,0.573989,0.608517,0.603561,0.575430,0.528675,0.455099,0.362264,0.347618,0.123781,0.022384,-0.061763,-0.121092,-0.146612,-0.153667,-0.136011,-0.145176,-0.127882,-0.136113,-0.140241,-0.140846,-0.146350,-0.136724,-0.154821,-0.183343
2,4,-50,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-78,-65,-999,-76,-999,-999,-999,-999,-999,-58,-999,-999,-999,-999,...,-0.169668,-0.187995,-0.189608,-0.188078,-0.180466,-0.167868,-0.157777,-0.153837,-0.165943,-0.193560,-0.210154,-0.229311,-0.244802,-0.243998,-0.231601,-0.210144,-0.206158,-0.170119,-0.158427,-0.155377,-0.169411,-0.185260,-0.199459,-0.217468,-0.229275,-0.232931,-0.212915,-0.181617,-0.156793,-0.126817,-0.113446,-0.118332,-0.136755,-0.154561,-0.182386,-0.198651,-0.199322,-0.178371,-0.147836,-0.117947
3,4,-45,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-9

In [31]:
test_df

,Unnamed: 0,00185299269cdc4716cdf21eff72a6fad5023508,00222417c6a5de878ea19d77aeb18fde4ce7f7fc,002b0212765d15559b547a0dc9beeba4119358d6,0041d652fd1fff3809e48d09ef0677b3ffaddf7f,00441fa1391be3b17a7d0c027ca68d6020c8199d,0052ad55475581cea50138350b7aba14f6028c1b,0057edcb7495ad0ec1be22f1340c28cb1de1d652,00607a6db35815085afc29935fa9b61c6ebc8200,006f284d510ecf7143271b7ec9aa85b899a400d6,0073518b549105172f0cf5dad39a607546e94260,0075b097629f5e0c9fe27fde740c52facbc1ab8f,00772269f87fee5e6270a74fe53ffdc3b3dc58ae,007be35d44badbcbc7750495120019dd49e85c0e,00aa0c01396f788b5292065f9cfe5b91c0329a88,00ae92121694f5c9f394464ddd40b57fc190fdd5,00b7742b740c1ff86b3d765c902ab43346244218,00b8d29564854cd6f9eb09d059c771b6865a5fd9,00cbe1309b87c31f7b85e610489883b83452bc53,00ce0e74971354a3a65e53508178f50a69df64ae,00d6c7ddc5a7d039475fbee5c2cb6956001c0205,0102a8b2fc053eabbff5103c5eaef5ca116006ed,01151e3322065cdddcaa1223e1fe0cf07408ba0f,012bbf20bfe9ad6b7d6612c02cf7e16741b5621d,013e18fb366e9f08b87327d657796b4533ed2e87,014150a311dc82924475bd2ff7745f2a51882df1,0141e36d66116a43423abe15837954c71f0585c6,0142cf843ab2b27ee1d0e50d27d864b380930908,01464a7834b3669524dc7c8a49d124cdcee3236d,014e4aba0e4667af7531dddee6df25f352d7afd1,01504013219f9be18f7dd995681848cbe2a033fb,0158d654aa7c09f75c333f5b5d2cce42711de4d1,015c643cc4abd226c32fb26327f6bddffe3654bb,01600f414415e3e1026c404503467d95652f3b74,0161da1c06de3894e20e2cda306093d57da3dd45,0169b9fdb8345c205d3f5130deb08bc674e1f1bc,017221f4dad519bf37a9b2e0d1d1238a70c07dd1,017803fcfb6227a2e7bab3a5e235b551480a0e53,017e93af3f4ecbcba0eaf965bd600bc3791f75d3,017f662f98112fbbd788e0312d80bd1050e1c1cd,...,fea8237dcb8eca2aba0b2e3ee6e31279ade4cf72,fea90fc6de1764e0b6962609caf34adf65064776,feba2b5033ff6f0bbdd88c35549e82256d4fba63,febb429d1d9589a984f515c3bafa0b0d915891bc,febefdca66c39d1f6192b3e25262e3a11564f3b9,fec3fd9162663227dea861510141235e24953d16,fecc2123f8c9a4b588292cd7c7a7ecec26cbd149,fed51a58fbe8ccf85fd0b2d23cbbd0521d4c0e93,fef596bbf2b83fb0ef265f57b24a63a932ffeb44,fef68964c729d7557366e950741c68e233b388d2,fef8ee158ce6a2e4187a3dacec0c989e60928baa,fefbc0692ba5d86cd913e94bfa51b3871c6689d1,ff0435c605070322f887f3d984f8f642ff4a49e8,ff088be7a5d004e16d9a5094597dab403b7b5bc5,ff0a2edfaea11fa2d0fa63ffe53633f2b9f054a8,ff15855c71397478a6427f4420f4e000819e7df1,ff2705d4018ca555305693fedab8e6d7f2e53b26,ff2c51f5007a9d4b9e8fb2953775a9b9c27660ee,ff37502265955c83f3624c6c295ede0f05747c85,ff3a5b95f4c988910f7145d0ceee2e04b8b693cb,ff3cf2e667bebd4826f0bb20f306d92ff9c528e0,ff48f7b6249ae90786e9caba3ef652a79189ba06,ff51b22ed833d0d5a77e30d3258e6495de86d373,ff51d3cc2690241d7843197883d901b63f48b3a8,ff548753b5012796024a0cbe180b1b0b7351399c,ff5509d3f6dcb91517ae7ad8159ee6a8b2728f67,ff66e4c0e09456272773aada3e75e337ccc16838,ff77bb2313288e6a1b03e701d685dd228f7ac1be,ff8caa92966872c41e43e6816a4fa7566352408e,ffafde33f629622f555f52a26e60117b5188ffe0,ffb93f1680174557263d813c52a460b51bd2e58a,ffc3cac3a8a07b7a31633302aaff237d50b676a7,ffe1992941063d9114d3533ac58453c56ba5a9b3,ffe6849ae196c548d0fe723c1d662db1f15e7011,ffe7eab32be71cead23850ff33d777f58f9738ae,ffec4048b9238a69d3894cd2ab3ef3b112f3de37,fff1bde1528b0849178fc349dbf20b1f15cf066e,site_path_timestamp,path,timestamp
0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-69,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,-999,-999,-999,-999,5d27075f03f801723c2e360f_108d983b84def2bd48bdc...,108d983b84def2bd48bdca36,10
1,4,-999,-999,-999,-999,-999,-999,-84,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-68,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,-999

In [35]:
test_sensor_df.iloc[:, 7033:7090]

,acc_acc_z_0,acc_acc_z_1,acc_acc_z_2,acc_acc_z_3,acc_acc_z_4,acc_acc_z_5,acc_acc_z_6,acc_acc_z_7,acc_acc_z_8,acc_acc_z_9,acc_acc_z_10,acc_acc_z_11,acc_acc_z_12,acc_acc_z_13,acc_acc_z_14,acc_acc_z_15,acc_acc_z_16,acc_acc_z_17,acc_acc_z_18,acc_acc_z_19,acc_acc_z_20,acc_acc_z_21,acc_acc_z_22,acc_acc_z_23,acc_acc_z_24,acc_acc_z_25,acc_acc_z_26,acc_acc_z_27,acc_acc_z_28,acc_acc_z_29,acc_acc_z_30,acc_acc_z_31,acc_acc_z_32,acc_acc_z_33,acc_acc_z_34,acc_acc_z_35,acc_acc_z_36,acc_acc_z_37,acc_acc_z_38,acc_acc_z_39,acc_acc_z_40,acc_acc_z_41,acc_acc_z_42,acc_acc_z_43,acc_acc_z_44,acc_acc_z_45,acc_acc_z_46,acc_acc_z_47,acc_acc_z_48,acc_acc_z_49,acc_timestamp,acc_path_id,acc_z_0,acc_z_1,acc_z_2,acc_z_3,acc_z_4
0,9.549377,10.640435,10.456160,10.882924,10.669511,8.736832,8.833544,9.581169,7.874676,7.324276,7.564445,8.073677,8.795202,7.920457,6.627857,5.786320,4.638581,6.955105,13.299736,18.953443,27.515829,25.757153,15.846028,10.482013,7.714214,5.564835,4.790756,3.370058,3.835814,4.785792,4.837355,3.902008,3.948829,4.564328,4.368921,3.882075,10.146057,33.162689,32.650754,17.570275,11.452200,12.240557,9.917733,4.312992,1.932329,1.725297,-0.072612,-1.324950,2.846295,4.665167,10.0,108d983b84def2bd48bdca36,-0.150696,-0.168618,-0.227297,-0.174179,-0.245951
1,4.841782,22.655035,6.223133,3.225105,18.705352,8.691144,5.931711,10.395158,11.370283,10.537376,6.368604,18.666565,7.538473,8.704414,10.021123,9.948902,9.863154,10.774483,8.539324,9.033586,10.176772,11.127876,8.039876,8.380014,13.425124,12.447185,8.138695,7.886765,9.348897,12.093624,9.586659,7.505433,9.626557,13.766523,11.106002,7.592463,8.462214,9.461497,13.844314,8.162767,8.706448,11.052445,9.040403,9.908781,9.671009,12.334668,7.814477,7.301860,8.128875,14.541395,3439.0,108d983b84def2bd48bdca36,-0.155487,0.736065,0.800063,0.053960,-0.163551
2,10.897919,7.635102,7.236864,11.249085,11.188531,7.402772,8.409041,14.642251,9.527965,7.913679,8.754086,13.033449,10.163381,7.072647,9.778782,13.974291,9.948281,8.253718,9.018795,13.469733,9.227694,8.125764,9.248047,12.779348,9.569362,7.735147,10.291393,12.524203,8.998058,8.035271,11.533698,9.405096,9.384515,10.058322,9.367896,9.939514,9.759632,9.715321,6.703555,10.310725,11.057737,9.665372,5.684712,7.568053,12.219028,9.331770,8.636152,7.750744,9.940991,7.531154,12526.0,108d983b84def2bd48bdca36,0.034668,-0.065610,-0.134491,-0.129940,0.037503
3,7.752533,9.249179,17.877343,11.946221,5.280093,6.386726,10.002180,15.824863,9.024497,7.831456,8.331814,9.740747,11.579875,9.611053,7.762645,9.894867,11.198579,9.387452,9.583455,10.137743,10.609040,11.857668,10.021077,10.249762,6.026393,13.723328,11.880977,7.789380,8.812460,12.780387,10.909244,6.596518,7.790222,5.457776,13.784498,7.197384,6.951486,11.440240,10.423927,7.994515,7.850197,15.261924,9.332630,7.777546,8.552947,15.531194,8.254254,6.556949,8.369978,15.731505,16915.0,108d983b84def2bd48bdca36,0.221619,0.711703,0.223166,0.906913,0.506535
4,9.321930,7.554777,7.778155,14.759362,5.688931,3.891637,26.696206,5.379859,3.421833,20.316544,7.798740,2.221485,14.857082,7.189065,8.600105,16.710277,8.380359,8.816817,9.953764,10.117727,10.847492,8.112972,9.533532,11.617735,9.286444,8.330117,10.345494,9.681192,10.355933,9.865302,10.017205,9.917258,9.825458,9.614470,6.686729,10.249265,9.461469,8.031560,8.680793,8.225505,10.815229,10.253538,11.577717,9.736051,8.596391,11.217232,9.172871,9.738810,9.930722,10.565361,25641.0,108d983b84def2bd48bdca36,0.201385,-0.018472,-0.092034,-0.006799,-0.370739
5,9.478165,8.842448,10.226694,11.362596,8.122857,9.457223,16.393761,7.328330,8.745771,10.802592,9.042647,10.152947,11.631955,9.490739,8.583373,8.169400,10.080974,8.912925,10.720896,9.892850,9.937940,9.991821,10.311977,12.089447,10.231822,9.958444,8.796701,11.521949,9.405386,7.849015,11.890635,10.427958,7.222840,9.835954,12.755743,8.755077,7.757562,13.282427,8.151676,7.888150,14.706299,8.649385,6.673842,14.110537,9.112108,5.806048,16.073297,9.104713,7.319832,13.623376,31577.0,108d983b84def2bd48bdca36,0.076218,0.201003,0.358140,-0.013687,

In [8]:
sensor_df

,way_Unnamed: 0,way_timestamp,way_x,way_y,way_path_id,acc_acc_z_0,acc_acc_z_1,acc_acc_z_2,acc_acc_z_3,acc_acc_z_4,acc_acc_z_5,acc_acc_z_6,acc_acc_z_7,acc_acc_z_8,acc_acc_z_9,acc_acc_z_10,acc_acc_z_11,acc_acc_z_12,acc_acc_z_13,acc_acc_z_14,acc_acc_z_15,acc_acc_z_16,acc_acc_z_17,acc_acc_z_18,acc_acc_z_19,acc_acc_z_20,acc_acc_z_21,acc_acc_z_22,acc_acc_z_23,acc_acc_z_24,acc_acc_z_25,acc_acc_z_26,acc_acc_z_27,acc_acc_z_28,acc_acc_z_29,acc_acc_z_30,acc_acc_z_31,acc_acc_z_32,acc_acc_z_33,acc_acc_z_34,...,acc_z_10,acc_z_11,acc_z_12,acc_z_13,acc_z_14,acc_z_15,acc_z_16,acc_z_17,acc_z_18,acc_z_19,acc_z_20,acc_z_21,acc_z_22,acc_z_23,acc_z_24,acc_z_25,acc_z_26,acc_z_27,acc_z_28,acc_z_29,acc_z_30,acc_z_31,acc_z_32,acc_z_33,acc_z_34,acc_z_35,acc_z_36,acc_z_37,acc_z_38,acc_z_39,acc_z_40,acc_z_41,acc_z_42,acc_z_43,acc_z_44,acc_z_45,acc_z_46,acc_z_47,acc_z_48,acc_z_49
0,0,1573878909217,215.11792,217.61235,5dcf885b94e49000061256c8,9.549377,10.640435,10.456160,10.882924,10.669511,8.736832,8.833544,9.581169,7.874676,7.324276,7.564445,8.073677,8.795202,7.920457,6.627857,5.786320,4.638581,6.955105,13.299736,18.953443,27.515829,25.757153,15.846028,10.482013,7.714214,5.564835,4.790756,3.370058,3.835814,4.785792,4.837355,3.902008,3.948829,4.564328,4.368921,...,0.715278,0.712194,0.710279,0.709076,0.707808,0.706451,0.704994,0.703608,0.699787,0.687446,0.670436,0.660028,0.668872,0.688236,0.707980,0.722882,0.733272,0.743938,0.755098,0.763294,0.766324,0.765245,0.762242,0.758299,0.753390,0.746249,0.731493,0.695477,0.670471,0.683299,0.701812,0.717276,0.723291,0.724743,0.726553,0.728979,0.730552,0.732138,0.735267,0.737083
1,1,1573878911613,212.32953,217.83267,5dcf885b94e49000061256c8,4.841782,22.655035,6.223133,3.225105,18.705352,8.691144,5.931711,10.395158,11.370283,10.537376,6.368604,18.666565,7.538473,8.704414,10.021123,9.948902,9.863154,10.774483,8.539324,9.033586,10.176772,11.127876,8.039876,8.380014,13.425124,12.447185,8.138695,7.886765,9.348897,12.093624,9.586659,7.505433,9.626557,13.766523,11.106002,...,0.798892,0.791074,0.787592,0.793101,0.793891,0.795581,0.799320,0.803759,0.816928,0.862403,0.920497,0.969894,0.990707,0.997353,0.122897,-0.995597,-0.991612,-0.982857,-0.970328,-0.960603,-0.954676,-0.950291,-0.950885,-0.955218,-0.960580,-0.965282,-0.968159,-0.969006,-0.965939,-0.965172,-0.961158,-0.961496,-0.964428,-0.963796,-0.961835,-0.968666,-0.972365,-0.974434,-0.976582,-0.976950
2,2,1573878914332,209.82498,218.13461,5dcf885b94e49000061256c8,10.897919,7.635102,7.236864,11.249085,11.188531,7.402772,8.409041,14.642251,9.527965,7.913679,8.754086,13.033449,10.163381,7.072647,9.778782,13.974291,9.948281,8.253718,9.018795,13.469733,9.227694,8.125764,9.248047,12.779348,9.569362,7.735147,10.291393,12.524203,8.998058,8.035271,11.533698,9.405096,9.384515,10.058322,9.367896,...,-0.982060,-0.981499,-0.983063,-0.983277,-0.982771,-0.982689,-0.984021,-0.984466,-0.984339,-0.983841,-0.984824,-0.984974,-0.984373,-0.982968,-0.984295,-0.984420,-0.983643,-0.981524,-0.983009,-0.983200,-0.983113,-0.983459,-0.983808,-0.983219,-0.983919,-0.984419,-0.985983,-0.993084,0.855607,0.988199,0.959439,0.922502,0.883153,0.869907,0.840799,0.808305,0.779713,0.777893,0.759689,0.756079
3,3,1573878920925,213.64041,212.79517,5dcf885b94e49000061256c8,7.752533,9.249179,17.877343,11.946221,5.280093,6.386726,10.002180,15.824863,9.024497,7.831456,8.331814,9.740747,11.579875,9.611053,7.762645,9.894867,11.198579,9.387452,9.583455,10.137743,10.609040,11.857668,10.021077,10.249762,6.026393,13.723328,11.880977,7.789380,8.812460,12.780387,10.909244,6.596518,7.790222,5.457776,13.784498,...,0.849051,0.859051,0.874928,0.891511,0.903738,0.907797,0.893036,0.856808,0.801818,0.747798,0.685923,0.602396,0.519410,0.476895,0.439369,0.413112,0.387749,0.374528,0.357233,0.330586,0.309658,0.300745,0.288315,0.293252,0.281458,0.305862,0.308804,0.312844,0.320515,0.322459,0.325801,0.326178,0.326756,0.322351,0.313838,0.306811,0.304992,0.300930,0.298187,0.302250
4,4,1573878925180,210.43849,209.35843,5dcf885b94e49000061256c8

In [99]:
a = pd.read_csv('/content/drive/MyDrive/acc_50/z/5a0546857ecc773753327266_train.csv')
b = pd.read_csv('/content/drive/MyDrive/rotation_50/z/5a0546857ecc773753327266_train.csv')
a.loc[:,['path_id','timestamp']]

,path_id,timestamp
0,5d10a1669c50c70008fe8977,1561368762838
1,5d10a1669c50c70008fe8977,1561368774057
2,5d10a1669c50c70008fe8977,1561368793535
3,5d10a1669c50c70008fe8977,1561368798851
4,5d10a1669c50c70008fe8977,1561368814782
...,...,...
2477,5e15bf91f4c3420006d52341,1578483574917
2478,5e15bf91f4c3420006d52341,1578483580134
2479,5e15bf941506f2000638fec5,1578483469045
2480,5e15bf941506f2000638fec5,1578483477240


In [100]:
b.loc[:,['path_id','timestamp']]

,path_id,timestamp
0,5d10a1669c50c70008fe8977,1561368762838
1,5d10a1669c50c70008fe8977,1561368774057
2,5d10a1669c50c70008fe8977,1561368793535
3,5d10a1669c50c70008fe8977,1561368798851
4,5d10a1669c50c70008fe8977,1561368814782
...,...,...
2477,5e15bf91f4c3420006d52341,1578483574917
2478,5e15bf91f4c3420006d52341,1578483580134
2479,5e15bf941506f2000638fec5,1578483469045
2480,5e15bf941506f2000638fec5,1578483477240


In [85]:
pd.concat(train_sensor_dfs)

AssertionError: ignored

In [80]:
train_sensor_dfs

KeyboardInterrupt: ignored

In [76]:
len(train_way_df['path_y'].unique())

511

In [71]:
train_way_df

,Unnamed: 0_x,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,x,y_x,f,path_x,timestamp_x,Unnamed: 0_y,timestamp_y,y_y,path_y
0,4,-999,-999,-999,-999,-999,-999,-999,-88,-999,-999,-999,-999,-86,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-90,-999,-999,-999,-999,-999,-999,-999,5.900912,93.41243,0,5e15b384f4c3420006d522e4,1578479746258,4,1578479748093,93.41243,5e15b384f4c3420006d522e4
1,4,-999,-999,-999,-999,-999,-999,-999,-90,-999,-999,-999,-999,-86,-999,-999,-999,-999,-999,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,5.900912,93.41243,0,5e15b3b2f4c3420006d522fe,1578479800506,4,1578479748093,93.41243,5e15b384f4c3420006d522e4
2,4,-999,-999,-999,-999,-999,-999,-999,-90,-999,-999,-999,-999,-89,-999,-999,-999,-999,-999,-91,-999,-999,-999,-999,-999,-9

In [ ]:
used_path = sorted(way_df['path'].value_counts().index.tolist())
    train_way_dfs = []
    for path in used_path:
        train_path_df = train_df[train_df['path'] == path].sort_values('x')
        way_path_df = way_df[way_df['path'] == path].sort_values('x')
        train_way_path_df = pd.merge_asof(train_path_df, way_path_df, on='x') 
        train_way_dfs.append(train_way_path_df)
    train_way_df = pd.concat(train_way_dfs)

In [66]:
train_way_path_df

,Unnamed: 0_x,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,x,y_x,f,path_x,timestamp_x,Unnamed: 0_y,timestamp_y,y_y,path_y
0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-68,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,78.218190,38.999737,2,5d10a1669c50c70008fe8977,1561368764807,0,1561368762838,38.999737,5d10a1669c50c70008fe8977
1,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-68,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,78.218190,38.999737,2,5d10a1669c50c70008fe8977,1561368766804,0,1561368762838,38.999737,5d10a1669c50c70008fe8977
2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-9

In [48]:
train = pd.read_csv('/content/drive/MyDrive/wifi_train_timestamp_test_path/5c3c44b80379370013e0fd2b_train.csv').sort_values('timestamp')
way = pd.read_csv('/content/drive/MyDrive/waypoint/5c3c44b80379370013e0fd2b_waypoint_train.csv').sort_values('timestamp')
acc = pd.read_csv('/content/drive/MyDrive/acc_50/z/5c3c44b80379370013e0fd2b_train.csv').fillna(0)

In [49]:
pd.merge_asof(train, way, on='timestamp')

,Unnamed: 0_x,0006968a29229c78478c7f6c2d2d37ee6ba145ef,000adfcfbf793ddc4f9c6a1a7c3bc78cffd01819,0018258e7c0f5223eb1750ada8ae2a4c4c58d21c,002c92111c356bc9fc054042cd2f0a96d9e43b06,0040b8059620d2c17e22e24abb76a679200aba6a,0041e6526da83cbdd4b171648c7f172fc325be85,0094988edb57e8605962a80834f8d45cdb7b5664,00b110f7f41db9767115afc45013c37cfc6b7f98,00e6c8b25a32bf8ef89c491daae09ee845ac450c,00f583061184d29cfac0e03cf0982edd025e677a,010c689afe822af414b2b35793afc26f7601fbd6,01303b1e8f97b01bad8797e315ed0a901a329be4,0141e500290392d659a7626bab8583cf8dc38e29,014495aeff4f7fc100db0e678170a63572b4697e,015c7c64b6a54349e2397de357f568512772fb3e,016eded3ba71cdb8878fff411645997d03694957,018ce87fe87decc1e5774da5dc34635f1184511e,018e923b2cceeb208de55aafffd91952cad67095,01a1425d2c391b1ce611307c5d9e2f5aae9d05a9,01c0147829c7633d7d4a5ff0f0fb7b9dcae47ddb,01ee8bd0434333056d99c09516b87057dcde31af,01f8283d6a9c4569645891e427a13424a64dea19,01fc007de38446168c161ccc468b600c3ca264a5,021633338aefbc95c57277a06ac248446877d7dc,0217dcc23e4d2dc353049039969395a1ad8f37de,02245fd2e9a7c1ed386ec3d898e5f396e8f06b7d,0249b5890e0f0ddf780785a396beece6917181a3,027817c190697bc6fadec198a06b280359eb4fe8,0290599d79e3d93d92925711de912b784674843e,029ef8e3586e31fc20cf72262a4769d71ad26c2e,029f08c9ded6f97b2d9835fb82e70353e86d3030,02b3f57280dda5a50d93e8b0859602e47d9c9a71,02b474371945f487190faaf51a03428644540af8,02cd4f9d65034f697b3cab4e57b316348f86d915,030a4d88c65cc68d4df9ca64051fb9d76381f87e,03271f9864b63a30538e1b00fa711d2a3d1ac62b,0363d84f19fc2a4b9990694d94f56b4c99ef06db,03ba19a589f9dcbaee0d67687dc32bccd0f4c96a,03bc00df43d070acb9cb598e152e3d015ad87499,...,fe1270bbeeda4af755073b81fcf963247ab37749,fe12ca15565f2be6f9305011009d68842d3017ed,fe1b648a92fd096d120f486263a9820ced8ec662,fe27059ab5ada4b759aae4ee1af721b11acc7c6a,fe284a652a33e96bfc5e0031b08aac28cfc1f073,fe41a82125e7e157183b4d618c7bd5ff5f5b6b4d,fe4bbd57142596850d13bc50cf610a39f69a5c65,fe65746a98a3688b8890916edaf9e1347ada8aa6,fe6862838f0772603d0f847b74ae54cc0281e683,fe6910534bac2f0600c5337c75861cd3f0f0dc4d,fe7b699db8097299c72c3c6ccbc62b8b8aa73fb9,fe8ae8201fd564a84414bf2e56fa093cca5f7062,fe931cb19ff4ecd2001edeccda55f33dbe795ee9,fe941f2eaa08adaea6cd8732507bbf12b63a10d7,fea332af9cc58eba9863547677d94decdf6b8ab1,feab3baf5e13f53a705be30bda7651bccbff14c2,feabcf093d8a7a22e3d1d83b7734a1ba9a39b0e2,febc8fd3c8509a5137227735ea327bfa2d9566f2,fef02e49b154db94fb12e1d6bf747fb3d3e7d18f,fef39546a10012e2cd11b0dd39b43f5c3b7068ee,fef8dfa758510940974763ac1680fd82a987a8a6,ff38f86ca36490f5beaa94621d1ef9b40d24fa4e,ff416e453cb71dd12c51bdeb90cd14537044a5b0,ff4b036abaffc8c0fc189ceb4352ddb4c999f1c2,ff4b64a9dcaca9fc11d1bd1b6c93ff03818dce86,ff63a0886a132b4d87891febc9d80f917d5145d8,ff686cc9f43c58b768dc20165f6e4903d403d599,ff9d5a42379c7712c1a557b22fba49271757d272,ffb4a2e1ea58ddc214706ce4bb9a0b62418c17bc,ffbbc3e5b5569397a31ba6a6b40a282722f2d601,ffe57a1050c12fb060adae004f8b2a5ebe9ba9bd,x_x,y_x,f,path,timestamp,Unnamed: 0_y,x_y,y_y,path_id
0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-60,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-65,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-82,161.49619,39.399570,0,5d08a2553f461f0008dac591,1560495551493,0,161.49619,39.39957,5d08a2553f461f0008dac591
1,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-60,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-64,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-65,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-81,-999,-999,-999,-999,-999,-78,161.49619,39.399570,0,5d08a2553f461f0008dac591,1560495553431,0,161.49619,39.39957,5d08a2553f461f0008dac591
2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-60,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-64,

In [53]:
train.head(10)

,Unnamed: 0,0006968a29229c78478c7f6c2d2d37ee6ba145ef,000adfcfbf793ddc4f9c6a1a7c3bc78cffd01819,0018258e7c0f5223eb1750ada8ae2a4c4c58d21c,002c92111c356bc9fc054042cd2f0a96d9e43b06,0040b8059620d2c17e22e24abb76a679200aba6a,0041e6526da83cbdd4b171648c7f172fc325be85,0094988edb57e8605962a80834f8d45cdb7b5664,00b110f7f41db9767115afc45013c37cfc6b7f98,00e6c8b25a32bf8ef89c491daae09ee845ac450c,00f583061184d29cfac0e03cf0982edd025e677a,010c689afe822af414b2b35793afc26f7601fbd6,01303b1e8f97b01bad8797e315ed0a901a329be4,0141e500290392d659a7626bab8583cf8dc38e29,014495aeff4f7fc100db0e678170a63572b4697e,015c7c64b6a54349e2397de357f568512772fb3e,016eded3ba71cdb8878fff411645997d03694957,018ce87fe87decc1e5774da5dc34635f1184511e,018e923b2cceeb208de55aafffd91952cad67095,01a1425d2c391b1ce611307c5d9e2f5aae9d05a9,01c0147829c7633d7d4a5ff0f0fb7b9dcae47ddb,01ee8bd0434333056d99c09516b87057dcde31af,01f8283d6a9c4569645891e427a13424a64dea19,01fc007de38446168c161ccc468b600c3ca264a5,021633338aefbc95c57277a06ac248446877d7dc,0217dcc23e4d2dc353049039969395a1ad8f37de,02245fd2e9a7c1ed386ec3d898e5f396e8f06b7d,0249b5890e0f0ddf780785a396beece6917181a3,027817c190697bc6fadec198a06b280359eb4fe8,0290599d79e3d93d92925711de912b784674843e,029ef8e3586e31fc20cf72262a4769d71ad26c2e,029f08c9ded6f97b2d9835fb82e70353e86d3030,02b3f57280dda5a50d93e8b0859602e47d9c9a71,02b474371945f487190faaf51a03428644540af8,02cd4f9d65034f697b3cab4e57b316348f86d915,030a4d88c65cc68d4df9ca64051fb9d76381f87e,03271f9864b63a30538e1b00fa711d2a3d1ac62b,0363d84f19fc2a4b9990694d94f56b4c99ef06db,03ba19a589f9dcbaee0d67687dc32bccd0f4c96a,03bc00df43d070acb9cb598e152e3d015ad87499,...,fdba8f8ae00d07494ff16de862b6c402d4b17f02,fde6334849758fdf471cd0779cd90e65525bd0d4,fdf33027e471ee0fc7a425f5a645737f8c6443f6,fe0512fae19c9ab8def5d76fca5e8c3a26364ed0,fe1270bbeeda4af755073b81fcf963247ab37749,fe12ca15565f2be6f9305011009d68842d3017ed,fe1b648a92fd096d120f486263a9820ced8ec662,fe27059ab5ada4b759aae4ee1af721b11acc7c6a,fe284a652a33e96bfc5e0031b08aac28cfc1f073,fe41a82125e7e157183b4d618c7bd5ff5f5b6b4d,fe4bbd57142596850d13bc50cf610a39f69a5c65,fe65746a98a3688b8890916edaf9e1347ada8aa6,fe6862838f0772603d0f847b74ae54cc0281e683,fe6910534bac2f0600c5337c75861cd3f0f0dc4d,fe7b699db8097299c72c3c6ccbc62b8b8aa73fb9,fe8ae8201fd564a84414bf2e56fa093cca5f7062,fe931cb19ff4ecd2001edeccda55f33dbe795ee9,fe941f2eaa08adaea6cd8732507bbf12b63a10d7,fea332af9cc58eba9863547677d94decdf6b8ab1,feab3baf5e13f53a705be30bda7651bccbff14c2,feabcf093d8a7a22e3d1d83b7734a1ba9a39b0e2,febc8fd3c8509a5137227735ea327bfa2d9566f2,fef02e49b154db94fb12e1d6bf747fb3d3e7d18f,fef39546a10012e2cd11b0dd39b43f5c3b7068ee,fef8dfa758510940974763ac1680fd82a987a8a6,ff38f86ca36490f5beaa94621d1ef9b40d24fa4e,ff416e453cb71dd12c51bdeb90cd14537044a5b0,ff4b036abaffc8c0fc189ceb4352ddb4c999f1c2,ff4b64a9dcaca9fc11d1bd1b6c93ff03818dce86,ff63a0886a132b4d87891febc9d80f917d5145d8,ff686cc9f43c58b768dc20165f6e4903d403d599,ff9d5a42379c7712c1a557b22fba49271757d272,ffb4a2e1ea58ddc214706ce4bb9a0b62418c17bc,ffbbc3e5b5569397a31ba6a6b40a282722f2d601,ffe57a1050c12fb060adae004f8b2a5ebe9ba9bd,x,y,f,path,timestamp
2695,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-60,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-60,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-65,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-82,161.49619,39.399570,0,5d08a2553f461f0008dac591,1560495551493
2696,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-60,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-64,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-57,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-65,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-81,-999,-999,-999,-999,-999,-78,161.49619,39.399570,0,5d08a2553f461f0008dac591,1560495553431
2697,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999

In [54]:
way.head(10)

,Unnamed: 0,timestamp,x,y,path_id
520,0,1560495549579,161.49619,39.399570,5d08a2553f461f0008dac591
521,1,1560495574531,182.92952,29.322659,5d08a2553f461f0008dac591
1365,0,1560500995805,198.05206,93.971240,5d073b814a19c000086c558b
1366,1,1560501011427,180.07335,89.948960,5d073b814a19c000086c558b
1442,0,1560501353313,180.07335,89.948960,5d073b821a69370008bc5cf8
1443,1,1560501359897,181.34761,84.303024,5d073b821a69370008bc5cf8
1444,2,1560501366899,179.01929,79.741630,5d073b821a69370008bc5cf8
1445,3,1560501382632,168.45207,72.536415,5d073b821a69370008bc5cf8
1446,4,1560501407093,148.75590,58.221370,5d073b821a69370008bc5cf8
1447,5,1560501416118,142.25401,57.584200,5d073b821a69370008bc5cf8
